<a href="https://colab.research.google.com/github/eliasaraujo-dev/gemini-health-insight/blob/main/GeminiHealthInsight_AIchat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Instalação da biblioteca Python do Google Gemini**

In [1]:
!pip install -q google-generativeai

**Importação das bibliotecas necessárias**

In [2]:
import google.generativeai as genai
import json # Para formatar os dados para o prompt
# Futuramente, para a fun;áo de gerar múltiplos pacientes:
import random
# Futuramente, para a interface
# import ipywidgets as widgets
# from IPython.display import display, Markdown

In [3]:
import os
from google.colab import userdata

try:
  GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
  genai.configure(api_key=GOOGLE_API_KEY)
  print("API Key configurada com sucesso!")
except userdata.SecretNotFoundError:
  print("Erro: Secret 'GEMINI_API_KEY' não encontrado. Por favor, configure-o no Colab.")
except Exception as e:
  print(f"Ocorreu um erro ao configurar a API Key: {e}")

API Key configurada com sucesso!


In [4]:
model = genai.GenerativeModel('gemini-2.0-flash')
print("Modelo Gemini carregado com sucesso!")

Modelo Gemini carregado com sucesso!


In [5]:
# Dados de um Único Paciente
paciente_exemplo_1 = {
    "id_paciente": "P001",
    "dados_pessoais": { # Adicionando alguns dados demográficos básicos
        "idade": 52,
        "sexo_biologico": "Masculino"
    },
    "dados_genomicos": {
        "Variante_Genetica_Colesterol": "Presente",
        "Polimorfismo_APOE": "e3/e4"
    },
    "historico_medico": {
        "Historico_Familiar_DCV": "Sim",
        "Parentesco_DCV": "Pai",
        "Idade_Diagnostico_Familiar_DCV": 55,
        "Diagnostico_Hipertensao": "Sim", # Modificado para mais risco
        "Pressao_Arterial_Sistolica_mmHg": 145, # Adicionado
        "Pressao_Arterial_Diastolica_mmHg": 92,  # Adicionado
        "Diagnostico_Diabetes": "Não",
        "Nivel_Colesterol_Ultimo_Exame": {
            "LDL_Colesterol_mg_dL": 150,
            "HDL_Colesterol_mg_dL": 45,
            "Triglicerideos_mg_dL": 180
        },
        "Medicacoes_Uso": ["Lisinopril 10mg"] # Medicamento para hipertensão
    },
    "estilo_de_vida": {
        "Alimentacao_Tipo": "Ocidental_Processada",
        "Frequencia_Alimentacao_Processada": "Alta",
        "Atividade_Fisica_Semanal_Minutos": 60,
        "Atividade_Fisica_Tipo": "Caminhada Leve",
        "Tabagismo_Status": "Ex-Fumante", # Modificado
        "Anos_Desde_Cessacao_Tabagismo": 5, # Adicionado
        "Consumo_Alcool_Frequencia": "Semanal", # Modificado
        "Consumo_Alcool_Doses_Por_Vez": 2, # Adicionado
        "Nivel_Estresse_Percebido": "Alto", # Modificado
        "Horas_Sono_Media_Noite": 6.0
    },
    "dados_ambientais": {
        "Cidade_Residencia": "Rio de Janeiro",
        "Bairro_Residencia_Simulado": "Centro", # Para simular exposição
        "Qualidade_Ar_Regiao_RJ": "Moderada",
        "Exposicao_Poluicao_Sonora": "Alta"
    }
}

# print(json.dumps(paciente_exemplo_1, indent=2, ensure_ascii=False)) # Imprime formatado

In [6]:
print(json.dumps(paciente_exemplo_1, indent=2, ensure_ascii=False))

{
  "id_paciente": "P001",
  "dados_pessoais": {
    "idade": 52,
    "sexo_biologico": "Masculino"
  },
  "dados_genomicos": {
    "Variante_Genetica_Colesterol": "Presente",
    "Polimorfismo_APOE": "e3/e4"
  },
  "historico_medico": {
    "Historico_Familiar_DCV": "Sim",
    "Parentesco_DCV": "Pai",
    "Idade_Diagnostico_Familiar_DCV": 55,
    "Diagnostico_Hipertensao": "Sim",
    "Pressao_Arterial_Sistolica_mmHg": 145,
    "Pressao_Arterial_Diastolica_mmHg": 92,
    "Diagnostico_Diabetes": "Não",
    "Nivel_Colesterol_Ultimo_Exame": {
      "LDL_Colesterol_mg_dL": 150,
      "HDL_Colesterol_mg_dL": 45,
      "Triglicerideos_mg_dL": 180
    },
    "Medicacoes_Uso": [
      "Lisinopril 10mg"
    ]
  },
  "estilo_de_vida": {
    "Alimentacao_Tipo": "Ocidental_Processada",
    "Frequencia_Alimentacao_Processada": "Alta",
    "Atividade_Fisica_Semanal_Minutos": 60,
    "Atividade_Fisica_Tipo": "Caminhada Leve",
    "Tabagismo_Status": "Ex-Fumante",
    "Anos_Desde_Cessacao_Tabagismo": 

In [7]:
# Função para Gerar Múltiplos Pacientes Simulados (Opcional, mas útil para testes)
def gerar_paciente_simulado(paciente_id):
    sexos = ["Masculino", "Feminino"]
    variantes_colesterol = ["Presente", "Ausente"]
    apo_e_tipos = ["e3/e3", "e3/e4", "e2/e3", "e4/e4"]
    sim_nao = ["Sim", "Não"]
    tipos_alimentacao = ["Ocidental_Processada", "Equilibrada_Mediterranea", "Vegetariana", "Baixo_Carboidrato"]
    niveis_atividade = ["Sedentário", "Caminhada Leve", "Atividade Moderada", "Atividade Intensa"]
    status_tabagismo = ["Não Fumante", "Ex-Fumante", "Fumante Atual"]
    frequencia_alcool = ["Nenhum", "Ocasional", "Semanal", "Diário"]
    niveis_estresse = ["Baixo", "Médio", "Alto"]
    qualidade_ar_bairros_rj = {
        "Centro": "Moderada", "Copacabana": "Moderada", "Tijuca": "Ruim",
        "Barra da Tijuca": "Boa", "Bangu": "Ruim", "Santa Cruz": "Moderada"
    }
    bairros = list(qualidade_ar_bairros_rj.keys())

    bairro_escolhido = random.choice(bairros)

    paciente = {
        "id_paciente": f"P{str(paciente_id).zfill(3)}",
        "dados_pessoais": {
            "idade": random.randint(30, 75),
            "sexo_biologico": random.choice(sexos)
        },
        "dados_genomicos": {
            "Variante_Genetica_Colesterol": random.choice(variantes_colesterol),
            "Polimorfismo_APOE": random.choice(apo_e_tipos)
        },
        "historico_medico": {
            "Historico_Familiar_DCV": random.choice(sim_nao),
            "Diagnostico_Hipertensao": random.choice(sim_nao),
            "Pressao_Arterial_Sistolica_mmHg": random.randint(110, 180) if random.choice(sim_nao) == "Sim" else None,
            "Pressao_Arterial_Diastolica_mmHg": random.randint(70, 110) if random.choice(sim_nao) == "Sim" else None,
            "Diagnostico_Diabetes": random.choice(sim_nao),
            "Nivel_Colesterol_Ultimo_Exame": {
                "LDL_Colesterol_mg_dL": random.randint(70, 250),
                "HDL_Colesterol_mg_dL": random.randint(30, 100),
                "Triglicerideos_mg_dL": random.randint(50, 400)
            },
            "Medicacoes_Uso": [f"Medicação Simulada {i+1}" for i in range(random.randint(0,3))]
        },
        "estilo_de_vida": {
            "Alimentacao_Tipo": random.choice(tipos_alimentacao),
            "Frequencia_Alimentacao_Processada": random.choice(["Baixa", "Média", "Alta"]),
            "Atividade_Fisica_Semanal_Minutos": random.choice([0, 30, 60, 90, 120, 150, 200, 300]),
            "Atividade_Fisica_Tipo": random.choice(niveis_atividade),
            "Tabagismo_Status": random.choice(status_tabagismo),
            "Anos_Desde_Cessacao_Tabagismo": random.randint(1, 20) if "Ex-Fumante" else None,
            "Consumo_Alcool_Frequencia": random.choice(frequencia_alcool),
            "Consumo_Alcool_Doses_Por_Vez": random.randint(1,5) if frequencia_alcool != "Nenhum" else 0,
            "Nivel_Estresse_Percebido": random.choice(niveis_estresse),
            "Horas_Sono_Media_Noite": round(random.uniform(4.5, 9.0), 1)
        },
        "dados_ambientais": {
            "Cidade_Residencia": "Rio de Janeiro",
            "Bairro_Residencia_Simulado": bairro_escolhido,
            "Qualidade_Ar_Regiao_RJ": qualidade_ar_bairros_rj[bairro_escolhido],
            "Exposicao_Poluicao_Sonora": random.choice(["Baixa", "Média", "Alta"])
        }
    }
    return paciente

# Gerar uma lista de pacientes simulados
# numero_de_pacientes_simulados = 5
# lista_pacientes_simulados = [gerar_paciente_simulado(i+1) for i in range(numero_de_pacientes_simulados)]

# Para visualizar o primeiro paciente gerado pela função:
# print("\nPaciente gerado pela função:")
# print(json.dumps(lista_pacientes_simulados[0], indent=2, ensure_ascii=False))

In [8]:
# Gerar uma lista de pacientes simulados
numero_de_pacientes_simulados = 5
lista_pacientes_simulados = [gerar_paciente_simulado(i+1) for i in range(numero_de_pacientes_simulados)]

In [9]:
# Para visualizar o primeiro paciente gerado pela função:
print("\nPaciente gerado pela função:")
print(json.dumps(lista_pacientes_simulados[0], indent=2, ensure_ascii=False))


Paciente gerado pela função:
{
  "id_paciente": "P001",
  "dados_pessoais": {
    "idade": 47,
    "sexo_biologico": "Feminino"
  },
  "dados_genomicos": {
    "Variante_Genetica_Colesterol": "Presente",
    "Polimorfismo_APOE": "e3/e4"
  },
  "historico_medico": {
    "Historico_Familiar_DCV": "Sim",
    "Diagnostico_Hipertensao": "Sim",
    "Pressao_Arterial_Sistolica_mmHg": 135,
    "Pressao_Arterial_Diastolica_mmHg": null,
    "Diagnostico_Diabetes": "Não",
    "Nivel_Colesterol_Ultimo_Exame": {
      "LDL_Colesterol_mg_dL": 221,
      "HDL_Colesterol_mg_dL": 33,
      "Triglicerideos_mg_dL": 65
    },
    "Medicacoes_Uso": [
      "Medicação Simulada 1",
      "Medicação Simulada 2"
    ]
  },
  "estilo_de_vida": {
    "Alimentacao_Tipo": "Ocidental_Processada",
    "Frequencia_Alimentacao_Processada": "Alta",
    "Atividade_Fisica_Semanal_Minutos": 150,
    "Atividade_Fisica_Tipo": "Atividade Intensa",
    "Tabagismo_Status": "Não Fumante",
    "Anos_Desde_Cessacao_Tabagismo": 8

In [10]:
# Imprimindo pacientes através do índice
print(lista_pacientes_simulados[0])

{'id_paciente': 'P001', 'dados_pessoais': {'idade': 47, 'sexo_biologico': 'Feminino'}, 'dados_genomicos': {'Variante_Genetica_Colesterol': 'Presente', 'Polimorfismo_APOE': 'e3/e4'}, 'historico_medico': {'Historico_Familiar_DCV': 'Sim', 'Diagnostico_Hipertensao': 'Sim', 'Pressao_Arterial_Sistolica_mmHg': 135, 'Pressao_Arterial_Diastolica_mmHg': None, 'Diagnostico_Diabetes': 'Não', 'Nivel_Colesterol_Ultimo_Exame': {'LDL_Colesterol_mg_dL': 221, 'HDL_Colesterol_mg_dL': 33, 'Triglicerideos_mg_dL': 65}, 'Medicacoes_Uso': ['Medicação Simulada 1', 'Medicação Simulada 2']}, 'estilo_de_vida': {'Alimentacao_Tipo': 'Ocidental_Processada', 'Frequencia_Alimentacao_Processada': 'Alta', 'Atividade_Fisica_Semanal_Minutos': 150, 'Atividade_Fisica_Tipo': 'Atividade Intensa', 'Tabagismo_Status': 'Não Fumante', 'Anos_Desde_Cessacao_Tabagismo': 8, 'Consumo_Alcool_Frequencia': 'Nenhum', 'Consumo_Alcool_Doses_Por_Vez': 3, 'Nivel_Estresse_Percebido': 'Alto', 'Horas_Sono_Media_Noite': 7.7}, 'dados_ambientais': 

**Análise de Risco Cardiovascular**

In [11]:
# Preparação do Prompt de Análise:
# Escolha um paciente para análise (o exemplo 1 ou um da lista gerada)
paciente_para_analise = paciente_exemplo_1 # Ou: lista_pacientes_simulados[0]

# Convertendo o dicionário do paciente para uma string JSON formatada para o prompt
dados_paciente_json_string = json.dumps(paciente_para_analise, indent=2, ensure_ascii=False)

prompt_analise_risco = f"""
Você é um assistente de análise de saúde preventiva.
Analise os seguintes dados de um indivíduo e identifique os principais fatores de risco cardiovascular.
Considere a combinação de fatores genômicos, histórico médico, estilo de vida e dados ambientais.
Forneça um resumo conciso dos fatores de risco identificados.
Não forneça aconselhamento médico, apenas identifique os fatores de risco com base nos dados.

Dados do Indivíduo:
{dados_paciente_json_string}

Análise de Risco Cardiovascular:
"""
# print("\nPrompt para Análise de Risco:")
# print(prompt_analise_risco)

In [12]:
print("\nPrompt para Análise de Risco:")
print(prompt_analise_risco)


Prompt para Análise de Risco:

Você é um assistente de análise de saúde preventiva.
Analise os seguintes dados de um indivíduo e identifique os principais fatores de risco cardiovascular.
Considere a combinação de fatores genômicos, histórico médico, estilo de vida e dados ambientais.
Forneça um resumo conciso dos fatores de risco identificados.
Não forneça aconselhamento médico, apenas identifique os fatores de risco com base nos dados.

Dados do Indivíduo:
{
  "id_paciente": "P001",
  "dados_pessoais": {
    "idade": 52,
    "sexo_biologico": "Masculino"
  },
  "dados_genomicos": {
    "Variante_Genetica_Colesterol": "Presente",
    "Polimorfismo_APOE": "e3/e4"
  },
  "historico_medico": {
    "Historico_Familiar_DCV": "Sim",
    "Parentesco_DCV": "Pai",
    "Idade_Diagnostico_Familiar_DCV": 55,
    "Diagnostico_Hipertensao": "Sim",
    "Pressao_Arterial_Sistolica_mmHg": 145,
    "Pressao_Arterial_Diastolica_mmHg": 92,
    "Diagnostico_Diabetes": "Não",
    "Nivel_Colesterol_Ultimo_

In [13]:
# Envio para a API e Recebimento da Resposta:
print(f"\nEnviando dados do paciente {paciente_para_analise['id_paciente']} para análise de risco cardiovascular...")
try:
    response_analise = model.generate_content(prompt_analise_risco)
    # print("\nResposta da Análise de Risco (Gemini):")
    # Para exibir de forma mais legível no Colab, especialmente se tiver markdown:
    # from IPython.display import Markdown
    # Markdown(response_analise.text)
    print(response_analise.text) # Impressão simples
except Exception as e:
    print(f"Ocorreu um erro ao gerar conteúdo: {e}")
    # print(f"Detalhes do erro: {response_analise.prompt_feedback}") # Para depurar bloqueios de segurança


Enviando dados do paciente P001 para análise de risco cardiovascular...
Com base nos dados fornecidos, os principais fatores de risco cardiovascular identificados para o paciente P001 são:

*   **Idade:** 52 anos (a idade aumenta o risco cardiovascular).
*   **Sexo:** Masculino (homens geralmente apresentam maior risco em idades mais jovens).
*   **Genética:**
    *   Variante Genética de Colesterol: Presente (predisposição genética para níveis elevados de colesterol).
    *   Polimorfismo APOE: e3/e4 (associado a um risco aumentado de doença cardiovascular e Alzheimer).
*   **Histórico Médico:**
    *   Histórico Familiar de DCV: Sim (pai diagnosticado aos 55 anos indica predisposição familiar).
    *   Hipertensão: Sim (pressão arterial sistólica e diastólica elevadas: 145/92 mmHg).
    *   Nível de Colesterol: LDL elevado (150 mg/dL) e HDL relativamente baixo (45 mg/dL), triglicerídeos elevados (180 mg/dL).
*   **Estilo de Vida:**
    *   Alimentação: Tipo ocidental processada e al

In [14]:
# Iteração e Refinamento de Prompts:


**Preparação do Prompt de Recomendações:**

In [15]:
# Preparação do Prompt de Recomendações:
# (Assumindo que 'paciente_para_analise' e 'dados_paciente_json_string' ainda estão definidos)

prompt_recomendacoes = f"""
Você é um assistente de saúde preventiva com foco em medicina de precisão.
Com base nos seguintes dados de um indivíduo e na análise de risco cardiovascular, gere recomendações preventivas personalizadas.
As recomendações devem ser práticas e acionáveis, focando em:
1. Mudanças no estilo de vida (dieta, exercício, sono, gerenciamento de estresse).
2. Sugestões de exames de rastreamento ou acompanhamento (com a ressalva de sempre consultar um médico).
3. Informações sobre como os fatores ambientais (ex: qualidade do ar no Rio de Janeiro) podem impactar sua saúde e possíveis mitigações.
4. Se aplicável, mencionar a importância de discutir predisposições genéticas com um profissional de saúde.

Importante: As recomendações NÃO substituem o aconselhamento médico profissional. Sempre indique que o usuário deve consultar um médico para decisões de saúde.

Dados do Indivíduo:
{dados_paciente_json_string}

Recomendações Preventivas Personalizadas:
"""
# print("\nPrompt para Recomendações:")
# print(prompt_recomendacoes)

In [16]:
# Código para Obter e Exibir Recomendações:
print(f"\nEnviando dados do paciente {paciente_para_analise['id_paciente']} para geração de recomendações no Gemini...")
try:
    response_recomendacoes = model.generate_content(prompt_recomendacoes)
    print("\nResposta das Recomendações (Gemini):")
    # from IPython.display import Markdown # Certifique-se que está importado
    # Markdown(response_recomendacoes.text)
    print(response_recomendacoes.text)
except Exception as e:
    print(f"Ocorreu um erro ao gerar conteúdo: {e}")
    # print(f"Detalhes do erro: {response_recomendacoes.prompt_feedback}")


Enviando dados do paciente P001 para geração de recomendações no Gemini...

Resposta das Recomendações (Gemini):
## Recomendações Preventivas Personalizadas para P001

**Importante:** As recomendações a seguir são baseadas nos dados fornecidos e NÃO substituem o aconselhamento médico profissional. Consulte um médico para discutir suas necessidades individuais e tomar decisões sobre sua saúde.

**Resumo da Análise de Risco:**

P001, um homem de 52 anos, apresenta múltiplos fatores de risco cardiovascular, incluindo histórico familiar de doença cardiovascular (DCV), hipertensão (já medicada), níveis elevados de LDL-colesterol e triglicerídeos, alimentação inadequada, baixa atividade física, estresse elevado, sono insuficiente e exposição à poluição ambiental no Rio de Janeiro. A variante genética relacionada ao colesterol e o polimorfismo APOE e3/e4 também contribuem para o risco. Embora seja um ex-fumante, a cessação do tabagismo ocorreu há relativamente pouco tempo.

**Recomendações D

**Integração de Dados Ambientais (Mais Detalhes)**

**Interface Simples no Google Colab com ipywidgets**

In [17]:
# Importar bibliotecas
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output # clear_output para limpar saídas anteriores

In [26]:
# Criação de Campos de Entrada
style = {'description_width': 'initial'} # Para que as descrições não sejam cortadas

idade_widget = widgets.IntText(description="Sua Idade:", value=50, style=style)
sexo_widget = widgets.Dropdown(options=["Masculino", "Feminino"], description="Sexo Biológico:", style=style)
ldl_widget = widgets.IntText(description="Seu LDL (mg/dL):", value=130, style=style)
hdl_widget = widgets.IntText(description="Seu HDL (mg/dL):", value=50, style=style)
hipertensao_widget = widgets.Checkbox(description="Diagnóstico de Hipertensão?", value=False, indent=False)
atividade_widget = widgets.Dropdown(
    options=[
        ("Sedentário (0 min/sem", 0),
        ("Leve (até 60min/sem)", 60),
        ("Moderada (até 150 min/sem)", 150),
        ("Intensa (>150 min/sem)", 200)
    ],
    description="Atividade Física Semanal:",
    style=style
)
bairro_widget = widgets.Dropdown(
    options=["Centro", "Copacabana", "Tijuca", "Barra da Tijuca", "Bangu", "Outro"],
    value="Centro",
    description="Seu Bairro (Rio de Janeiro):",
    style=style
)

# Exibir os widgets
# display(idade_widget, sexo_widget, ldl_widget, hdl_widget, hipertensao_widget, atividade_widget, bairro_widget)

In [27]:
botao_analisar = widgets.Button(description="Analisar e Gerar Recomendações")
output_area = widgets.Output() # Área para exibir os resultados

def on_button_clicked(b):
    with output_area:
        clear_output(wait=True) # Limpa a saída anterior
        print("Coletando dados e preparando para análise...")

        # Construir um dicionário de paciente com os dados dos widgets
        # (Esta é uma simplificação, você precisaria mapear para a estrutura completa do paciente)
        dados_usuario_widgets = {
            "id_paciente": "WidgetUser001",
            "dados_pessoais": {
                "idade": idade_widget.value,
                "sexo_biologico": sexo_widget.value
            },
            "historico_medico": {
                "Diagnostico_Hipertensao": "Sim" if hipertensao_widget.value else "Não",
                "Nivel_Colesterol_Ultimo_Exame": {
                    "LDL_Colesterol_mg_dL": ldl_widget.value,
                    "HDL_Colesterol_mg_dL": hdl_widget.value,
                    "Triglicerideos_mg_dL": 150 # Valor fixo para simplificar
                },
                # Adicione mais campos conforme necessário
            },
            "estilo_de_vida": {
                "Atividade_Fisica_Semanal_Minutos": atividade_widget.value,
                # Adicione mais campos
            },
            "dados_ambientais": {
                "Cidade_Residencia": "Rio de Janeiro",
                "Bairro_Residencia_Simulado": bairro_widget.value,
                "Qualidade_Ar_Regiao_RJ": "Moderada" # Simplificado, poderia buscar com base no bairro
            }
            # Adicione dados genômicos simulados fixos ou outras simplificações
        }
        dados_usuario_json_string = json.dumps(dados_usuario_widgets, indent=2, ensure_ascii=False)
        print("Dados coletados. Enviando para o Gemini...")

        prompt_final_usuario = f"""
        Você é um assistente de saúde preventiva.
        Analise os seguintes dados de um indivíduo e gere recomendações preventivas personalizadas,
        focando em risco cardiovascular, dieta, exercícios, e considerando o ambiente no Rio de Janeiro.
        Lembre-se de que as recomendações não substituem o aconselhamento médico.

        Dados do Indivíduo:
        {dados_usuario_json_string}

        Análise e Recomendações:
        """

        try:
            # Reutilize o 'model' configurado no Passo 1
            response_widgets = model.generate_content(prompt_final_usuario)
            print("\n--- Análise e Recomendações do Gemini ---")
            display(Markdown(response_widgets.text))
        except Exception as e:
            print(f"Erro ao contatar o Gemini: {e}")
            # if 'response_widgets' in locals() and response_widgets.prompt_feedback:
            #    print(f"Feedback do prompt: {response_widgets.prompt_feedback}")


botao_analisar.on_click(on_button_clicked)

# Exibir todos os widgets e o botão
display(idade_widget, sexo_widget, ldl_widget, hdl_widget, hipertensao_widget, atividade_widget, bairro_widget, botao_analisar, output_area)

IntText(value=50, description='Sua Idade:', style=DescriptionStyle(description_width='initial'))

Dropdown(description='Sexo Biológico:', options=('Masculino', 'Feminino'), style=DescriptionStyle(description_…

IntText(value=130, description='Seu LDL (mg/dL):', style=DescriptionStyle(description_width='initial'))

IntText(value=50, description='Seu HDL (mg/dL):', style=DescriptionStyle(description_width='initial'))

Checkbox(value=False, description='Diagnóstico de Hipertensão?', indent=False)

Dropdown(description='Atividade Física Semanal:', options=(('Sedentário (0 min/sem', 0), ('Leve (até 60min/sem…

Dropdown(description='Seu Bairro (Rio de Janeiro):', options=('Centro', 'Copacabana', 'Tijuca', 'Barra da Tiju…

Button(description='Analisar e Gerar Recomendações', style=ButtonStyle())

Output()